In [ ]:
#@title **Ollama AI文本翻译/Ollama AI Translation:**

# @markdown **</br>**<font size="2"> 使用Ollama来执行本地部署的LLM，想要免费使用AI翻译的用户可以执行以下单元格。
# @markdown **</br>**阅读项目文档以了解更多。</font>
# @markdown **</br>**<font size="2"> Since Ollama provides the ability to deploy LLMs locally, the users that want to use free AI translations can execute the following blocks of code.
# @markdown **</br>**Then generate bilingual subtitle files in same sub style.Read documentaion to learn more.</font>

In [ ]:
#@title **下载Ollama Library/Importing Ollama Library**

!sudo apt update

!sudo apt install -y pciutils

!curl https://ollama.ai/install.sh | sh

In [ ]:
#@title **初始化Ollama Server/Starting Ollama Server**


import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [ ]:
#@title **下载Ollama LLM/Pulling Ollama LLM**

# @markdown **用户可以在这里选择想要部署的LLM**

# @markdown **The user can choose the LLM type for the deployment here**

model_type = "deepseek-r1:14b"  # @param ["llama3.2:3b", "deepseek-r1:7b", "deepseek-r1:14b", "deepseek-llm"]

! ollama pull $model_type

In [ ]:
#@title **进行翻译/Translating The Subtitles**

! pip install -U langchain-ollama

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

import sys
import os
import re
import time
import codecs
import regex as re
from pathlib import Path
from tqdm import tqdm
from google.colab import files
from IPython.display import clear_output

uploaded = files.upload()
sub_name = list(uploaded.keys())[0]
sub_basename = Path(sub_name).stem

clear_output()

! pip install pysubs2

import pysubs2

output_format = "ass"  # @param ["ass","srt"]

target_language = 'zh-hans'# @param ["zh-hans","english"]
template = "You are a language expert.Your task is to translate the input subtitle text, sentence by sentence, into {target_language}.However, please utilize the context to improve the accuracy and quality of translation.Please be aware that the input text could contain typos and grammar mistakes, utilize the context to correct the translation.Please return only translated content and do not include the origin text. Do not return your thinking process and only return the translated text. Please do not use any punctuation around the returned text.Please do not translate people's name and leave it as original language. Here is the text to translate: {text}\"" # @param {type:"string"}

# @markdown <font size="4">Default prompt: </br>
# @markdown ```You are a language expert.```</br>
# @markdown ```Your task is to translate the input subtitle text, sentence by sentence, into {target_language}.```</br>
# @markdown ```Please utilize the context to improve the accuracy and quality of translation.```</br>
# @markdown ```Please be aware that the input text could contain typos and grammar mistakes, utilize the context to correct the translation.```</br>
# @markdown ```Please return only translated content and do not include the origin text.```</br>
# @markdown ```Do not return your thinking process and only return the translated text.```</br>
# @markdown ```Please do not use any punctuation around the returned text.```</br>
# @markdown ```Please do not translate people's name and leave it as original language.```</br>
# @markdown ```Here is the text to translate: {text}```</br>

llm = OllamaLLM(model=model_type)
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

def translate(prompt, language, text):
        # print(text)
        # self.rotate_key()
        t_text = chain.invoke({"target_language": language, "text": text})
        print(t_text)
        if model_type == "deepseek-r1:14b" or model_type == "deepseek-r1:7b":
          return t_text.split("\n")[len(t_text.split("\n"))-1]
        else:
          return t_text

class SubtitleTranslator():
    def __init__(self, sub_src):
        self.sub_src = sub_src
        self.translations = []

    def translate_by_line(self):
        sub_trans = pysubs2.load(self.sub_src)
        total_lines = len(sub_trans)
        for line in tqdm(sub_trans,total = total_lines):
            line_trans = translate(prompt, target_language, line.text)
            line.text += (line_trans)
            print(line_trans)
            self.translations.append(line_trans)
        return sub_trans, self.translations

clear_output()

t = SubtitleTranslator(sub_src=sub_name)

translation, _, = t.translate_by_line()

if output_format == 'ass':
  translation.save(sub_basename + '_translation.ass')
  files.download(sub_basename + '_translation.ass')
elif output_format == 'srt':
  translation.save(sub_basename + '_translation.srt')
  files.download(sub_basename + '_translation.srt')

print('双语字幕生成完毕 All done!')